In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss

sys.path.insert(0, "../input/pytorch-lightning")

In [ ]:
# TabNet
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl pytorch-tabnet
# Iterative Stratification
!pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

# EfficientNet
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/gen-efficientnet-pretrained/tf_efficientnet_*.pth /root/.cache/torch/hub/checkpoints/
!ls -la /root/.cache/torch/hub/checkpoints/

!pip install ../input/gen-efficientnet-pytorch

In [ ]:
def create_submission():
    test = pd.read_csv('../input/lish-moa/test_features.csv')
    train_target_columns = pd.read_csv('../input/lish-moa/train_targets_scored.csv').columns

    submission = pd.DataFrame(columns=train_target_columns)
    submission.sig_id = test.sig_id
    submission.iloc[:, 1:] = 0.0

    return submission

def load_df_list(file_paths):
    df_list = []

    for p in file_paths:
        if type(p) is pd.DataFrame:
            df_list.append(p)
        else:
            df_list.append(pd.read_csv(p))
        
    df1 = df_list[0]
    
    for (i, df2) in enumerate(df_list[1:]):
        assert len(df1) == len(df2)
        
        df_list[i + 1] = df2.set_index('sig_id').loc[df1.sig_id].reset_index()
          
    return df_list

def apply_emsamble(df_list, weights=None):
    return np.average([df.iloc[:, 1:].values for df in df_list], axis=0, weights=weights)

def create_ensamble_submission(paths, weights=None):
    submission = create_submission()

    df_list = load_df_list(paths)
    ensamble = apply_emsamble(df_list, weights=weights)
    
    submission.iloc[:, 1:] = ensamble
    
    print('Check NA: ', submission.isna().any().any())
    
    return submission

In [ ]:
!python ../input/prediction-nn-tabnet/sub_dae_3.py

In [ ]:
!python ../input/prediction-nn-tabnet/prediction-01-nn.py
!python ../input/prediction-nn-tabnet/prediction-02-tabnet_2.py
!python ../input/prediction-deepinsight/prediction-03-deepinsight_v3.py

In [ ]:
os.listdir('.')

In [ ]:
submission = create_ensamble_submission([
    'blending-02-tabnet-submission.csv',
    'blending-01-nn-0-submission.csv',
    'blending-01-nn-1-submission.csv',
    'blending-01-nn-2-submission.csv',
    'blending-01-nn-3-submission.csv',
    'blending-01-nn-4-submission.csv',
    'blending-01-nn-5-submission.csv',
    'deepinsignt-submission_0.csv',
    'deepinsignt-submission_1.csv',
], weights=(0.5, 0.7, 0.7, 0.5, 1.5, 1.5, 0.5, 0.4, 0.5))

submission.to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head()